# Tool to compute genus generating series for H-linear and H-circular families

**Author**: Wenjie Fang

**Date**: 2025-07-07

**License**: GPLv3

We are given a graph $H = (V, E)$ with two non-intersecting subsets $U_1, U_2$ of nodes, and a gluing function $h: U_1 \to U_2$. We want to gind the genus generating series of the $H$-linear and $H$-circular families. By Euler's formula, we may instead work with the number of faces. We will mark the number of copies by $t$, and the number of faces by $x$. At the end, we will reconvert these generating functions to ones that mark the genus.

In [1]:
Rx.<x> = PolynomialRing(ZZ) # for genus
Rtx.<t> = PowerSeriesRing(Rx) # for size
Polytx = PolynomialRing(Rx, t)
Fractx = Polytx.fraction_field()

## Representation of graphs

We will assume that the graph $H$ is connected, and we represent it as input by a list of its edges, each a pair of integers representing its $n$ nodes running from $0$ to $n-1$. For instance, the graph with two vertices and a triple edge will be ``[(0, 1), (0, 1), (0, 1)]``.

A dart is a pair of a node and one of its adjacent edge. We give the darts of a given node consecutive numbers. The embeddings are represented by two permutations over the darts, an involution $\tau$ without fixed point representing which two darts share the same node, and a general permutation $\sigma$ representing the order of darts around each node.

In our implementation, the numbering of darts is fixed in advanced, thus $\tau$ is fixed. We only need to go over all possible $\sigma$, which has a fixed cycle type.

In [2]:
def get_deg_seq(G, n):
    '''
    Returns a list storing the degree of each node
    '''
    degseq = [0] * n
    for (a, b) in G:
        degseq[a] += 1
        degseq[b] += 1
    return degseq

def get_dart_indices(G, n):
    '''
    returns the starting index of darts associated to each node.
    '''
    curflag = [1] * n
    degseq = get_deg_seq(G, n)
    for i in range(1, n):
        curflag[i] = curflag[i - 1] + degseq[i - 1]
    return curflag

def get_edge_perm_list(G, n):
    '''
    returns the involution without fixed point for the given graph, as a list of pairs
    '''
    pl = []
    # compute starting half-edge indices
    dartidx = get_dart_indices(G, n)
    # construct the list
    for e in G:
        darts = []
        for i in range(2):
            dart = dartidx[e[i]]
            dartidx[e[i]] += 1
            darts.append(dart)
        pl.append(tuple(darts))
    return pl

def compute_graph_genus_poly(G, n):
    '''
    Compute the genus polynomial of the given graph G with n nodes.
    '''
    m = len(G) # number of edges
    # compute the permutation for edges
    edgeperm = sage.combinat.permutation.from_cycles(2 * m, get_edge_perm_list(G, n))
    # get the needed information
    degseq = get_deg_seq(G, n)
    dartidx = get_dart_indices(G, n)
    # construct cycles for each node
    cycsets = [CyclicPermutations(range(dartidx[i], dartidx[i] + degseq[i])) for i in range(n)]
    # cartesian product of everyone
    accu = [0] * (m - n + 2)
    for cycles in sage.misc.mrange.cartesian_product_iterator(cycsets):
        nodeperm = sage.combinat.permutation.from_cycles(2 * m, cycles)
        accu[m - n - (nodeperm * edgeperm).cycle_type().length() + 2] += 1
    return sum([accu[a] * t^a for a in range(m - n + 2)])

compute_graph_genus_poly([(0,1), (0,1), (0,1)], 2)

2 + 2*t^2

## Partial face distribution

In the process, we need to compute the face element of $H_{*\varphi}$ restricted to darts related to nodes in $U_1 \cup U_2$, formally $\Phi^{D(H)}_{D^H}(\mathbf{R}(H_{*\varphi}))$. For this, we need the following function that computes this restricted face element.

In [3]:
def compute_face_element_proj(G, n, vl):
    '''
    Compute the face element of the given graph, blown-up at a given list of nodes, restricted to the darts of those nodes.
    
    The result is given as a dictionary with face permutation as key and the polynomial (monomial in this case) as value,
    along with a list of the list of darts associated to each one of the nodes.
    '''
    m = len(G) # number of edges
    # get the needed information
    degseq = get_deg_seq(G, n)
    dartidx = get_dart_indices(G, n)
    # construct cycles for each node, distinguishing those node whose permutation we keep, excluding the kept vertices
    cycsets = [list(CyclicPermutations(range(dartidx[i], dartidx[i] + degseq[i]))) for i in range(n) if i not in vl]
    # construct the group algebra that we are working with
    partialelem = [e for i in vl for e in range(dartidx[i], dartidx[i] + degseq[i])]
    PSG = SymmetricGroup(partialelem)
    # compute the permutation for edges
    edgeperm = sage.combinat.permutation.from_cycles(2 * m, get_edge_perm_list(G, n))
    # product then projection of everyone
    accufinal = {}
    for cycles in sage.misc.mrange.cartesian_product_iterator(cycsets):
        nodeperm = sage.combinat.permutation.from_cycles(2 * m, cycles)
        faceperm = Permutation(edgeperm * nodeperm).cycle_tuples()
        # count faces involving only darts not in the list
        facecnt = len(tuple(1 for f in faceperm if all((i not in partialelem) for i in f)))
        # projection for permutation
        faceperm = tuple(tuple(i for i in f if i in partialelem) for f in faceperm)
        faceperm = tuple(f for f in faceperm if f)
        faceperm = PSG(faceperm)
        if faceperm not in accufinal:
            accufinal[faceperm] = 0
        accufinal[faceperm] += x^facecnt
    return accufinal, [list(range(dartidx[i], dartidx[i] + degseq[i])) for i in vl]

compute_face_element_proj([(0, 1), (1, 2), (2, 3), (3, 4)] * 2, 5, [0, 4])

({(1,15,2,16): 4,
  (1,15,16,2): 4*x^2 + 16,
  (1,2,15,16): 4*x^2 + 16,
  (1,2,16,15): 4*x^2 + 16,
  (1,16,15,2): 4*x^2 + 16,
  (1,16,2,15): 4,
  (1,15): 4,
  (1,15,16): 4*x,
  (15,16): 16,
  (1,16,15): 4*x,
  (1,16): 4,
  (2,15): 4,
  (2,15,16): 4*x,
  (2,16,15): 4*x,
  (2,16): 4,
  (1,15,2): 4*x,
  (1,2)(15,16): 48*x,
  (1,16,2): 4*x,
  (1,2): 16,
  (1,2,15): 4*x,
  (1,2,16): 4*x},
 [[1, 2], [15, 16]])

## Manual projection of darts

Other than computing the projection of the face element of $H$, we also need to do projection for some product results. Therefore, we need some functionnalities to compute the projection. In this end, we will do computation with SymmetricGroup, which allows us to specify the domain, as it would change from one to the other.

In [4]:
def perm_proj(perm, PSG):
    '''
    Project a permutation into the domain given by PSG by ignoring elements that are not in the domain in the cycle notation
    '''
    domain = PSG.domain()
    # count faces involving only darts not in the list
    facecnt = len(tuple(1 for f in perm.cycle_tuples(singletons=True) if all((i not in domain) for i in f)))
    return PSG(tuple(tuple(i for i in f if i in domain) for f in perm.cycle_tuples(singletons=True))), Rx(x)^facecnt

def perm_proj_opt(perm, PSG, domain):
    '''
    Project a permutation into the domain given by PSG by ignoring elements that are not in the domain in the cycle notation
    '''
    # count faces involving only darts not in the list
    facecnt = len(tuple(1 for f in perm.cycle_tuples(singletons=True) if all((i not in domain) for i in f)))
    return PSG(tuple(tuple(i for i in e if i in domain) for e in perm.cycle_tuples(singletons=True))), Rx(x)^facecnt

def group_alg_proj(algelem, domain):
    '''
    Same as perm_proj, but works on group algebra
    '''
    PSG = SymmetricGroup(domain)
    accu = {}
    for perm, val in algelem.items():
        pproj, power = perm_proj_opt(perm, PSG, domain)
        if pproj not in accu:
            accu[pproj] = 0
        accu[pproj] += power * val
    return accu

group_alg_proj(compute_face_element_proj([(0, 1), (1, 2), (2, 3)] * 2, 4, [0, 3])[0], [1, 2, 11])

{(1,11): 6, (2,11): 6, (1,11,2): 6*x, (1,2): 12, (1,2,11): 6*x}

As we are going to do vertex-amalgamation of graphs, for simplicity, we need to adjust from time to time the domain of the symmetric group involved so that the numbering of darts is not too big. However, this is optional.

In [5]:
def standardize_domain_alg(algelem, domain_shift):
    '''
    Change the domain of the given element in group algebra to be on a different range

    Return the element with the domain normalized, and a dictionary translating the elements of the old domain to those in the new one.
    '''
    for e in algelem:
        olddomain = list(e.parent().domain())
        break
    ddict = {}
    for i in range(len(olddomain)):
        ddict[olddomain[i]] = i + domain_shift
    PSG = SymmetricGroup(range(domain_shift, domain_shift + len(olddomain)))
    accu = {}
    for perm, val in algelem.items():
        newperm = PSG(tuple(tuple(ddict[i] for i in c) for c in perm.cycle_tuples()))
        accu[newperm] = val # no conflict is possible here
    return accu, ddict

We also need to multiply elements in the group algebra, which may be on different supports.

In [6]:
def multiply_algelem(elem1, elem2):
    '''
    Compute the product of two element in group algebras, which may have different support.
    
    The result is thus supported on the union of the supports
    '''
    # extraction of domains, and computing the new domain
    dom1, dom2 = [], []
    for e in elem1:
        dom1 = list(e.parent().domain())
        break
    for e in elem2:
        dom2 = list(e.parent().domain())
        break
    dom = dom1 + dom2
    PSG = SymmetricGroup(dom)
    # do the product, one by one
    accu = {}
    elem1 = {PSG(e): v for e, v in elem1.items()}
    elem2 = {PSG(e): v for e, v in elem2.items()}
    for e1, v1 in elem1.items():
        for e2, v2 in elem2.items():
            e = e1 * e2
            if e not in accu:
                accu[e] = 0
            accu[e] += v1 * v2
    return accu

for e in multiply_algelem(compute_face_element_proj([(0,1)] * 3, 2, [0, 1])[0], {SymmetricGroup([7, 8])(1): 7}):
    print(e.parent().domain())

{1, 2, 3, 4, 5, 6, 7, 8}


## Applying the operators in chain

We now need to construct a function that applies these operators in the correct order to obtain iteratively the face polynomial of the $H$-linear family. Then another function for the $H$-circular family by linking the two ends of graphs in the $H$-linear family. For this, we need a function that compute the face element of a vertex-amalgamation of two graphs, given their proper exploded.

In [7]:
def vertex_amalgamation(eG, eH, vG, vH):
    '''
    Computes the face element of the vertex amalgamation of G and H, given as face elements.

    More precisely, we suppose that eG (resp. eH) is the face element of the blow-up of G (resp. H) at vertices in vG (resp. vH).
    
    We assume that vG and vH are of the same length, and we will fusion vertices in these orders.

    We also automatically project the element obtained out of the darts in the fusioned vertices

    Alongside with the face element, we also return a dictionary on the darts *that are not merged*.
    '''
    if len(vG) != len(vH):
        raise ValueError("Not the same number of vertices in the two graphs")
    # standardize everything
    eG, ddictG = standardize_domain_alg(eG, 1)
    eH, ddictH = standardize_domain_alg(eH, max(ddictG.values()) + 1)
    # compute the full domain
    if ddictH:
        fulldom = [i + 1 for i in range(max(ddictH.values()))]
    elif ddictG:
        fulldom = [i + 1 for i in range(max(ddictG.values()))] # for the case of H empty used to undo blow-up
    else:
        fulldom = [] # really, nothing is left?
    vGnew = [[ddictG[n] for n in node] for node in vG]
    vHnew = [[ddictH[n] for n in node] for node in vH]
    # we only need to keep non-fusioned darts
    for node in vG:
        for n in node:
            ddictG.pop(n)
    for node in vH:
        for n in node:
            ddictH.pop(n)
    vG, vH = vGnew, vHnew
    # compute the reduced domain
    fdarts = [i for node in vG for i in node] + [i for node in vH for i in node]
    reddom = [i for i in fulldom if i not in fdarts]
    # compute the darts of fusioned vertices
    vfusion = [vG[i] + vH[i] for i in range(len(vG))]
    # preparation
    PSG = SymmetricGroup(fulldom)
    PSGred = SymmetricGroup(reddom)
    eG = {PSG(e): v for e, v in eG.items()}
    eH = {PSG(e): v for e, v in eH.items()}
    accu = {}
    cycsets = [CyclicPermutations(node) for node in vfusion]
    for nodecyc in sage.misc.mrange.cartesian_product_iterator(cycsets): # the element \mathbb{C}_{U}
        cu = PSG([tuple(cyc) for cyc in nodecyc]) # PSG only accept collection of tuples
        for elemG, valG in eG.items():
            tmp = cu * elemG
            for elemH, valH in eH.items():
                res, face = perm_proj_opt(tmp * elemH, PSGred, reddom)
                if res not in accu:
                    accu[res] = 0
                accu[res] += face * valG * valH
    return accu, ddictG, ddictH

In [8]:
# test

testH, testHnodes = compute_face_element_proj([(0, 1), (1, 2)] * 2, 3, [0, 2])
vertex_amalgamation(testH, testH, [testHnodes[1]], [testHnodes[0]])

({(2,7): 4,
  (2,7,8): 4*x,
  (7,8): 16,
  (2,8,7): 4*x,
  (2,8): 4,
  (1,2): 16,
  (1,2)(7,8): 48*x,
  (1,2,7): 4*x,
  (1,2,7,8): 4*x^2 + 16,
  (1,2,8,7): 4*x^2 + 16,
  (1,2,8): 4*x,
  (1,7,2): 4*x,
  (1,7,8,2): 4*x^2 + 16,
  (1,8,7,2): 4*x^2 + 16,
  (1,8,2): 4*x,
  (1,7): 4,
  (1,7,8): 4*x,
  (1,8,7): 4*x,
  (1,8): 4,
  (1,7,2,8): 4,
  (1,8,2,7): 4},
 {1: 1, 2: 2},
 {7: 7, 8: 8})

In [9]:
compute_face_element_proj([(0, 1), (1, 2), (2, 3), (3, 4)] * 2, 5, [0, 4])

({(1,15,2,16): 4,
  (1,15,16,2): 4*x^2 + 16,
  (1,2,15,16): 4*x^2 + 16,
  (1,2,16,15): 4*x^2 + 16,
  (1,16,15,2): 4*x^2 + 16,
  (1,16,2,15): 4,
  (1,15): 4,
  (1,15,16): 4*x,
  (15,16): 16,
  (1,16,15): 4*x,
  (1,16): 4,
  (2,15): 4,
  (2,15,16): 4*x,
  (2,16,15): 4*x,
  (2,16): 4,
  (1,15,2): 4*x,
  (1,2)(15,16): 48*x,
  (1,16,2): 4*x,
  (1,2): 16,
  (1,2,15): 4*x,
  (1,2,16): 4*x},
 [[1, 2], [15, 16]])

We also need a function that closes up the blown-up vertices. This can be seen as a vertex-amalgamation with a graph with the correct number of vertices but not any edges.

In [10]:
def undo_blowup_face_element(eG, vG):
    '''
    We assume that all darts are bound after undoing the blow-up
    '''
    return vertex_amalgamation(eG, {SymmetricGroup([])(): 1}, vG, [[] for _ in vG])[0][SymmetricGroup([])()]

undo_blowup_face_element(compute_face_element_proj([(0, 1), (1, 2), (2, 3), (3, 4)] * 2, 5, [0, 4])[0], [[1, 2], [15, 16]])

64*x^5 + 128*x^3 + 24*x

## Computation of face polynomials of H-linear and H-circular families

We are now prepared to compute the face polynomials of H-linear and H-circular families. Note that this is not yet the genus polynomial, and we will need an extra conversion for the genus polynomial.

In [11]:
def linear_face_poly_iter(H, n, U1, U2):
    '''
    An iterator that compute the face polynomial of a H-circular family, given by the following parameters:
    - pair H and n (number of vertices)
    - k as number of repetitions of the same pattern
    - U1 and U2 as gluing (from U1 to U2)
    '''
    # some sanity checks
    if len(U1) != len(U2) or len(set(U1 + U2)) != len(U1) * 2: # same size and no intersection
        raise ValueError('Specification of gluing incorrect')
    # compute the elementary brick of blowup of H
    blowupH, vblowupH = compute_face_element_proj(H, n, U1 + U2)
    vU1, vU2 = vblowupH[:len(U1)], vblowupH[len(U1):]
    # start iteration
    curG, curU1, curU2 = blowupH, vU1, vU2
    yield 1
    yield undo_blowup_face_element(curG, curU1 + curU2)
    while True:
        # do amalgamation
        curG, ddictG, ddictH = vertex_amalgamation(curG, blowupH, curU1, vU2)
        # rename darts using the dictionary
        curU2 = [[ddictG[n] for n in node] for node in curU2]
        curU1 = [[ddictH[n] for n in node] for node in vU1]
        yield undo_blowup_face_element(curG, curU1 + curU2)
    return

def linear_face_poly_list(H, n, U1, U2, k):
    '''
    Compute the face polynomial of a H-circular family, given by the following parameters:
    - pair H and n (number of vertices)
    - k as number of repetitions of the same pattern
    - U1 and U2 as gluing (from U1 to U2)
    '''
    accu = []
    iter = linear_face_poly_iter(H, n, U1, U2)
    for _ in range(k + 1):
        accu.append(next(iter))
    return accu

In [12]:
linear_face_poly_list([[0, 1]] * 3, 2, [1], [0], 5)

[1,
 2*x^3 + 2*x,
 36*x^5 + 300*x^3 + 144*x,
 648*x^7 + 11880*x^5 + 34992*x^3 + 10080*x,
 11664*x^9 + 330480*x^7 + 2275776*x^5 + 3585600*x^3 + 708480*x,
 209952*x^11 + 8048160*x^9 + 91585728*x^7 + 339655680*x^5 + 340174080*x^3 + 49766400*x]

In [13]:
def circular_face_poly_iter(H, n, U1, U2):
    '''
    An iterator that computes the face polynomial of a H-circular family, given by the following parameters:
    - pair H and n (number of vertices)
    - k as number of repetitions of the same pattern
    - U1 and U2 as gluing (from U1 to U2)
    '''
    # some sanity checks
    if len(U1) != len(U2) or len(set(U1 + U2)) != len(U1) * 2: # same size and no intersection
        raise ValueError('Specification of gluing incorrect')
    # compute the elementary brick of blowup of H
    blowupH, vblowupH = compute_face_element_proj(H, n, U1 + U2)
    vU1, vU2 = vblowupH[:len(U1)], vblowupH[len(U1):]
    # start iteration
    curG, curU1, curU2 = blowupH, vU1, vU2
    yield 1
    yield undo_blowup_face_element(curG, [curU1[i] + curU2[i] for i in range(len(curU1))])
    while True:
        # do amalgamation
        curG, ddictG, ddictH = vertex_amalgamation(curG, blowupH, curU1, vU2)
        # rename darts using the dictionary
        curU2 = [[ddictG[n] for n in node] for node in curU2]
        curU1 = [[ddictH[n] for n in node] for node in vU1]
        yield undo_blowup_face_element(curG, [curU1[i] + curU2[i] for i in range(len(curU1))])
    return accu

def circular_face_poly_list(H, n, U1, U2, k):
    '''
    Computes the face polynomial of a H-circular family, given by the following parameters:
    - pair H and n (number of vertices)
    - k as number of repetitions of the same pattern
    - U1 and U2 as gluing (from U1 to U2)
    '''
    accu = []
    iter = circular_face_poly_iter(H, n, U1, U2)
    for _ in range(k + 1):
        accu.append(next(iter))
    return accu

In [14]:
circular_face_poly_list([[0, 1]] * 3, 2, [1], [0], 5)

[1,
 40*x^4 + 80*x^2,
 120*x^6 + 4200*x^4 + 10080*x^2,
 216*x^8 + 37224*x^6 + 618624*x^4 + 1071936*x^2,
 1296*x^10 + 300672*x^8 + 11328336*x^6 + 90449568*x^4 + 105280128*x^2,
 7776*x^12 + 3281472*x^10 + 169921152*x^8 + 2721612960*x^6 + 12212327232*x^4 + 9776049408*x^2]

## Compute only the denominator

We know that the face series is rational in `x` and `t`. In practice, it may be easier to compute first the denominator, which can be expressed as the determinant of a matrix, then use it to multiply the first terms to get the numerator.

In [15]:
def dict_to_cycles(d):
    '''
    Auxiliary function. Converts a dictionary representing a permutation of arbitrary domain into a cycle representation.

    Note that the dictionary is consumed after the conversion.
    '''
    accu = []
    while d:
        # pick an arbitrary element
        start = 0
        for elem in d:
            start = elem
            break
        # find the cycle
        cycle = [start]
        nextelem = d[start]
        while nextelem != start:
            cycle.append(nextelem)
            nextelem = d[nextelem]
        accu.append(tuple(cycle))
        # remove the elements in the cycle
        for e in cycle:
            d.pop(e)
    return accu

def H_family_det(H, n, U1, U2):
    '''
    Compute the determinant of the matrix related to H-linear and H-circular families, always counting faces
    '''
    if len(U1) != len(U2):
        raise ValueError("Not the same number of vertices in the two graphs")
    # compute the face element of H
    eH, Ulist = compute_face_element_proj(H, n, U1 + U2)
    U1, U2 = Ulist[:len(U1)], Ulist[len(U1):]
    # standardize everything
    _,  ddictG = standardize_domain_alg(eH, 1)
    eH, ddictH = standardize_domain_alg(eH, max(ddictG.values()) + 1)
    U1G = [[ddictG[n] for n in node] for node in U1]
    U1H = [[ddictH[n] for n in node] for node in U1]
    U2G = [[ddictG[n] for n in node] for node in U2]
    U2H = [[ddictH[n] for n in node] for node in U2]
    # compute the domains
    fulldom = [i + Integer(1) for i in range(max(ddictH.values()))]
    reddom = [n for node in U1H + U2G for n in node]
    # preparation
    PSG = SymmetricGroup(fulldom)
    PSGred = SymmetricGroup(reddom)
    eH = {PSG(e): v for e, v in eH.items()}
    dartsG = [n for node in U1G + U2G for n in node]
    U1G_to_U1H = {ddictG[n]: ddictH[n] for node in U1 for n in node}
    accu = {}
    vfusion = [U1G[i] + U2H[i] for i in range(len(U1G))]
    cycsets = [CyclicPermutations(node) for node in vfusion]
    for nodecyc in sage.misc.mrange.cartesian_product_iterator(cycsets): # the element \mathbb{C}_{U}
        cu = PSG([tuple(cyc) for cyc in nodecyc]) # PSG only accept collection of tuples
        for elemGp in Permutations(dartsG):
            elemG = PSG(dict_to_cycles({dartsG[i]: elemGp[i] for i in range(len(reddom))}))
            elemGproj = PSGred(tuple(tuple((U1G_to_U1H[n] if n in U1G_to_U1H else n) for n in cyc) for cyc in elemG.cycle_tuples()))
            if elemGproj not in accu:
                accu[elemGproj] = {}
            tmp = cu * elemG
            for elemH, valH in eH.items():
                res, face = perm_proj_opt(tmp * elemH, PSGred, reddom)
                if res not in accu[elemGproj]:
                    accu[elemGproj][res] = 0
                accu[elemGproj][res] += face * valH
    # now convert to a matrix and compute
    elemlist = [PSGred(dict_to_cycles({reddom[i]: elem[i] for i in range(len(reddom))})) for elem in Permutations(reddom)]
    elemcnt = len(elemlist)
    elemdict = {elemlist[i]: i for i in range(len(elemlist))}
    coeffdict = {}
    for e1 in accu:
        idx1 = elemdict[e1]
        for e2 in accu[e1]:
            coeffdict[(idx1, elemdict[e2])] = accu[e1][e2]
    m = identity_matrix(Fractx, elemcnt) - matrix(Fractx, elemcnt, elemcnt, coeffdict) * Fractx(t)
    return m.determinant(algorithm='df').numerator() # we only have polynomial coefficients here, and df is division-free

In [16]:
H_family_det([(0, 1)] * 2, 2, [1], [0]).factor()

(-1) * (2*t - 1) * (2*t + 1) * (12*t^2 + 4*x*t - 1)^2 * (12*t^2 + 2*x*t - 1)^4

In [17]:
H_family_det([(0, 1)] * 2, 2, [1], [0]).factor() # todo

(-1) * (2*t - 1) * (2*t + 1) * (12*t^2 + 4*x*t - 1)^2 * (12*t^2 + 2*x*t - 1)^4

## Compute the generating function using the denominator

Now we want to use this denominator to compute the rational generating function for H-linear and H-circular families. The idea is that, as we can compute the determinant separately, it suffices to compute the first terms and multiply by this denominator to get the numerator, with an approximation issue, of course.

In [18]:
def H_linear_gf(H, n, U1, U2):
    '''
    Compute the face generating function of a given H-linear family using the determinant.
    '''
    # constant for the gap
    GAPLEN = 5
    # first compute the determinant
    det = H_family_det(H, n, U1, U2)
    detdeg = det.degree()
    # then compute the first terms of the gf
    polylist = linear_face_poly_list(H, n, U1, U2, detdeg + GAPLEN)
    polyfirst = sum(polylist[i] * t^i for i in range(1, len(polylist)))
    numer = det * Polytx(polyfirst)
    # find the gap in the numerator
    degs = set(x for x in numer.dict().keys())
    gap = set(range(1, max(degs) + 1)) - degs
    if len(gap) < GAPLEN:
        raise ValueError('Surprising error occured, not having the correct gap')
    maxdeg = min(gap)
    numerdict = numer.dict()
    numer = sum(Fractx(t)^i * numerdict[i] for i in numerdict if i < maxdeg)
    return numer / det

In [19]:
H_linear_gf([(0, 1)] * 2, 2, [1], [0])

(2*x*t^2 + x^2*t)/(-12*t^2 - 4*x*t + 1)

In [20]:
def H_circular_gf(H, n, U1, U2):
    '''
    Compute the face generating function of a given H-linear family using the determinant.
    '''
    # constant for the gap
    GAPLEN = 5
    # first compute the determinant
    det = H_family_det(H, n, U1, U2)
    detdeg = det.degree()
    # then compute the first terms of the gf
    polylist = circular_face_poly_list(H, n, U1, U2, detdeg + GAPLEN)
    polyfirst = sum(polylist[i] * t^i for i in range(1, len(polylist)))
    numer = det * Polytx(polyfirst)
    # find the gap in the numerator
    degs = set(x for x in numer.dict().keys())
    gap = set(range(1, max(degs) + 1)) - degs
    if len(gap) < GAPLEN:
        raise ValueError('Surprising error occured, not having the correct gap')
    maxdeg = min(gap)
    numerdict = numer.dict()
    numer = sum(Fractx(t)^i * numerdict[i] for i in numerdict if i < maxdeg)
    return numer / det

In [21]:
testgf = H_circular_gf([(0, 1)] * 2, 2, [1], [0])

In [22]:
testgf.numerator()

(288*x^4 + 288*x^2)*t^6 + (144*x^5 + 336*x^3 - 96*x)*t^5 + (16*x^6 + 80*x^4 - 240*x^2)*t^4 + (4*x^5 - 140*x^3 + 16*x)*t^3 + (-18*x^4 + 18*x^2)*t^2 + (4*x^3 + 2*x)*t

In [23]:
testgf.denominator().factor()

(-1) * (2*t - 1) * (2*t + 1) * (12*t^2 + 2*x*t - 1) * (12*t^2 + 4*x*t - 1)

## Heuristic computation of the generating function using Padé approximation

In general, the computation of the determinant takes quite some time. Furthermore, we see that in many cases the real denominator is but a factor of the determinant. Therefore, it may be more interesting to simply compute the first terms and then do a Padé approximation. Note that, however, this does not give a proof, as it is always possible that we don't have enough term. However, it is reasonable if there is a gap between the demanded degree and the degree of the results, as generically they always match when we don't have enough terms.

However, as we are dealing with Padé's approximation with potentially high degree for elements in $\mathbb{Z}[x][[t]$, we need some more effort to shorten the computation time. We have a two-step strategy:

- First, given the first terms of a series in $\mathbb{Z}[x]$, as the coefficients may be large, we project them into $\mathrm{GF}(p)[x]$ for a quicker computation, and reconstruct the coefficients by taking several different $p$ and then using CRT.
- Second, it still takes a lot of time to do Padé's approximation when the coefficients are polynomials, as the rational reconstruction need to work in the fraction field, meaning that we don't have control on the degrees. We then take different random values of $x$ to reduce the problem into Padé's approximation over polynomials in $\mathrm{GF}(p)[x]$, which is significantly faster, and then reconstruct the polynomial coefficients using Lagrange's interpolation.

All these steps are possible because we assume that the denominator of these rational generating functions has constant term $1$, which is combinatorial and corresponds to our cases here.

In [24]:
def fast_pade(polylistmod, deg_estimate, degnumer, degdenom, PSeries, PFrac):
    '''
    Perform a faster Padé approximation using interpolation
    '''
    xvals = []
    numervals = []
    denomvals = []
    PPoly = polylistmod[0].parent()
    for i in range(deg_estimate):
        randx = PPoly.base().random_element()
        while randx in xvals:
            randx = PPoly.base().random_element()
        xvals.append(randx)
        paderes = sum(polylistmod[i].substitute(x=randx) * PSeries(t) ** i for i in range(1, len(polylistmod))).pade(deg_estimate, deg_estimate)
        padenumer, padedenom = paderes.numerator(), paderes.denominator()
        cst = padedenom.constant_coefficient().inverse()
        numervals.append((padenumer * cst).coefficients(sparse=False))
        denomvals.append((padedenom * cst).coefficients(sparse=False))
    valcnt = len(xvals)
    ndegpoly = [PPoly.lagrange_polynomial([(xvals[i], numervals[i][ndeg]) for i in range(valcnt)], algorithm='neville') for ndeg in range(degnumer + 1)]
    ddegpoly = [PPoly.lagrange_polynomial([(xvals[i], denomvals[i][ddeg]) for i in range(valcnt)], algorithm='neville') for ddeg in range(degdenom + 1)]
    while True: # denominator seems ok, but the numerator does not converge
        randx = PPoly.base().random_element()
        while randx in xvals:
            randx = PPoly.base().random_element()
        xvals.append(randx)
        paderes = sum(polylistmod[i].substitute(x=randx) * PSeries(t) ** i for i in range(1, len(polylistmod))).pade(deg_estimate, deg_estimate)
        padenumer, padedenom = paderes.numerator(), paderes.denominator()
        cst = padedenom.constant_coefficient().inverse()
        numervals.append((padenumer * cst).coefficients(sparse=False))
        denomvals.append((padedenom * cst).coefficients(sparse=False))
        valcnt += 1
        Flag = True
        numerpoly, denompoly = 0, 0
        for ndeg in range(degnumer + 1):
            ndegpoly[ndeg] = PPoly.lagrange_polynomial([(xvals[i], numervals[i][ndeg]) for i in range(valcnt)], algorithm='neville', previous_row=ndegpoly[ndeg])
            if ndegpoly[ndeg][-1].degree() >= valcnt - 2:
                Flag = False
                break
            numerpoly += ndegpoly[ndeg][-1] * PSeries(t) ** ndeg
        for ddeg in range(degdenom + 1):
            ddegpoly[ddeg] = PPoly.lagrange_polynomial([(xvals[i], denomvals[i][ddeg]) for i in range(valcnt)], algorithm='neville', previous_row=ddegpoly[ddeg])
            if ddegpoly[ddeg][-1].degree() >= valcnt - 2:
                Flag = False
                break
            denompoly += ddegpoly[ddeg][-1] * PSeries(t) ** ddeg
        if Flag:
            break
    return PFrac(numerpoly) / PFrac(denompoly)

In [25]:
import time

def pade_heuristic(H, n, U1, U2, polyiter):
    '''
    Compute the face generating function of the H-circular or H-linear family using Padé's approximation, depending on the iterator given.

    This algorithm is as follows:
    - We first determine the likely degrees of numerator and denominator with a prime modulo; 
    - Then we perform the Padé approximation using the CRT, because it is much faster:
        - We add a prime modulo and compute the Padé approximation;
            - Here we use the `fast_pade` function that uses Lagrange interpolation to accelerate.
        - We use CRT to compute the coefficients of the numerator and the denominator;
        - We iterate until the coefficients stablize.
    '''
    deg_estimate = 8
    deg_increment = 1
    GAPLEN = 2
    polylist = []
    polyiter = polyiter(H, n, U1, U2) # we use a lazy way to avoid recomputing
    starttime = time.time()
    print('Estimation phase')
    while True:
        # compute the first terms of the gf
        polytime = time.time() # time spent on computing terms of series
        while len(polylist) <= deg_estimate * 2 + GAPLEN:
            polylist.append(next(polyiter))
            if len(polylist) < 21:
                print(f'Series terms count: {len(polylist)}, time elapsed {time.time() - starttime}')
        print(f'Series terms count: {len(polylist)}, time elapsed {time.time() - starttime}')
        polytime = time.time() - polytime
        # do the modulo reduction
        p = random_prime(2 ** 62, lbound=2 ** 61)
        PPoly = PolynomialRing(GF(p), x)
        PFrac = PolynomialRing(PPoly, t).fraction_field()
        PSeries = PowerSeriesRing(PPoly, t)
        polylistmod = [PPoly(e) for e in polylist]
        # pade for the modulo
        padetime = time.time() # time spent on pade approximation
        randx = PPoly.base().random_element()
        pademod = sum(polylistmod[i].substitute(x=randx) * PSeries(t) ** i for i in range(1, len(polylistmod))).pade(deg_estimate, deg_estimate)
        degnumer, degdenom = pademod.numerator().degree(), pademod.denominator().degree()
        if not (degnumer >= deg_estimate or degdenom >= deg_estimate):
            pademod = fast_pade(polylistmod, deg_estimate, degnumer, degdenom, PSeries, PFrac)
        padetime = time.time() - padetime
        if polytime < padetime * 8: # spending more time on pade approximation, better to compute a few more terms before redoing it
            deg_increment += 1
        degnumer, degdenom = pademod.numerator().degree(), pademod.denominator().degree()
        if degnumer >= deg_estimate or degdenom >= deg_estimate:
            print(f'Degrees: numerator {degnumer}, denominator {degdenom}, estimate {deg_estimate}, time {padetime}. Not enough gap, restart.')
            deg_estimate += deg_increment
        else:
            print(f'Degrees: numerator {degnumer}, denominator {degdenom}, estimate {deg_estimate}, time {padetime}. Enough gap, start full approximation by CRT.')
            break
    # now use Chinese remainder theorem to accelerate computation
    # to save time, first get the previous result
    print(f'Prime modulo: {p}')
    plist = [p]
    normalize = pademod.denominator().truncate(1).inverse() # we need to keep the constant term of the denominator to 1
    numerdict = {(i, j): ZZ(val * normalize) for i, poly in pademod.numerator().dict().items() for j, val in poly.dict().items()}
    numer_resdict = {p : [e] for p, e in numerdict.items()}
    denomdict = {(i, j): ZZ(val * normalize) for i, poly in pademod.denominator().dict().items() for j, val in poly.dict().items()}
    denom_resdict = {p : [e] for p, e in denomdict.items()}
    while True:
        # get a new modulo and compute
        p = random_prime(2 ** 62, lbound = 2 ** 61)
        print(f'Prime modulo: {p}')
        while p in plist:
            p = random_prime(2 ** 62, lbound = 2 ** 61)
        plist.append(p)
        pprod = product(plist)
        pprodx = product(plist[:-1])
        PPoly = PolynomialRing(GF(p), x)
        PFrac = PolynomialRing(PPoly, t).fraction_field()
        PSeries = PowerSeriesRing(PPoly, t)
        polylistmod = [PPoly(e) for e in polylist]
        pademod = fast_pade(polylistmod, deg_estimate, degnumer, degdenom, PSeries, PFrac)
        normalize = pademod.denominator().truncate(1).inverse()
        flag = False
        for i, poly in pademod.numerator().dict().items(): # any KeyError is simply bad chance...
            for j, val in poly.dict().items():
                numer_resdict[(i, j)].append(ZZ(val * normalize))
                coeff = CRT_list(numer_resdict[(i, j)], plist)
                if coeff != numerdict[(i, j)] and pprod - coeff != pprodx - numerdict[(i, j)]: # also for negative coefficients
                    flag = True
                numerdict[(i, j)] = coeff
        for i, poly in pademod.denominator().dict().items(): # any KeyError is simply bad chance...
            for j, val in poly.dict().items():
                denom_resdict[(i, j)].append(ZZ(val * normalize))
                coeff = CRT_list(denom_resdict[(i, j)], plist)
                if coeff != denomdict[(i, j)] and pprod - coeff != pprodx - denomdict[(i, j)]: # also for negative coefficients
                    flag = True
                denomdict[(i, j)] = coeff
        if not flag: # we stop when adding a new prime does not change anything
            break
    # convert negative coefficients
    for key in denomdict:
        if denomdict[key] * 2 > pprod:
            denomdict[key] -= pprod
    for key in numerdict:
        if numerdict[key] * 2 > pprod:
            numerdict[key] -= pprod
    numer = sum(Fractx(t) ** i * Rx.monomial(j) * numerdict[(i, j)] for (i, j) in numerdict)
    denom = sum(Fractx(t) ** i * Rx.monomial(j) * denomdict[(i, j)] for (i, j) in denomdict)
    print(f'Total time: {time.time() - starttime}')
    return numer, denom

def pade_heuristic_linear(H, n, U1, U2):
    '''
    Compute the face generating function of the H-linear family.

    A thin wrapper of `pade_heuristic`
    '''
    return pade_heuristic(H, n, U1, U2, linear_face_poly_iter)

def pade_heuristic_circular(H, n, U1, U2):
    '''
    Compute the face generating function of the H-circular family.

    A thin wrapper of `pade_heuristic`
    '''
    return pade_heuristic(H, n, U1, U2, circular_face_poly_iter)

In [26]:
testgfnumer, testgfdenom = pade_heuristic_linear([[0, 1]] * 3, 2, [1], [0])

Estimation phase
Series terms count: 1, time elapsed 0.0009214878082275391
Series terms count: 2, time elapsed 0.001374959945678711
Series terms count: 3, time elapsed 0.006846427917480469
Series terms count: 4, time elapsed 0.09459090232849121
Series terms count: 5, time elapsed 0.43059229850769043
Series terms count: 6, time elapsed 0.8864617347717285
Series terms count: 7, time elapsed 1.3595490455627441
Series terms count: 8, time elapsed 1.8433687686920166
Series terms count: 9, time elapsed 2.319758176803589
Series terms count: 10, time elapsed 2.7886459827423096
Series terms count: 11, time elapsed 3.2615280151367188
Series terms count: 12, time elapsed 3.742609977722168
Series terms count: 13, time elapsed 4.227165460586548
Series terms count: 14, time elapsed 4.798281669616699
Series terms count: 15, time elapsed 5.302072048187256
Series terms count: 16, time elapsed 5.84506630897522
Series terms count: 17, time elapsed 6.369824171066284
Series terms count: 18, time elapsed 6.

In [27]:
testgfdenom.factor()

43200*t^3 + (-1080*x^2 + 2880)*t^2 + (-18*x^2 - 120)*t + 1

In [28]:
testgfnumer, testgfdenom = pade_heuristic_circular([[0, 1]] * 3, 2, [1], [0])

Estimation phase
Series terms count: 1, time elapsed 0.0006945133209228516
Series terms count: 2, time elapsed 0.002741575241088867
Series terms count: 3, time elapsed 0.06463289260864258
Series terms count: 4, time elapsed 0.38547635078430176
Series terms count: 5, time elapsed 1.044654130935669
Series terms count: 6, time elapsed 1.819652795791626
Series terms count: 7, time elapsed 2.621429920196533
Series terms count: 8, time elapsed 3.4244940280914307
Series terms count: 9, time elapsed 4.220074653625488
Series terms count: 10, time elapsed 5.037166357040405
Series terms count: 11, time elapsed 5.865373373031616
Series terms count: 12, time elapsed 6.718021869659424
Series terms count: 13, time elapsed 7.57464861869812
Series terms count: 14, time elapsed 8.461743831634521
Series terms count: 15, time elapsed 9.365655183792114
Series terms count: 16, time elapsed 10.283171653747559
Series terms count: 17, time elapsed 11.235028743743896
Series terms count: 18, time elapsed 12.2112

In [29]:
testgfdenom.factor()

(-1) * (6*t + 1) * (12*t - 1) * (360*t^2 + (-12*x + 30)*t - 1) * (360*t^2 + (12*x + 30)*t - 1) * (43200*t^3 + (-1080*x^2 + 2880)*t^2 + (-18*x^2 - 120)*t + 1) * (259200*t^4 + (-2160*x^2 + 60480)*t^3 + (-612*x^2 + 2160)*t^2 + (-6*x^2 - 114)*t + 1)

## Last step: from face generating function to genus generating function

For simplicity, we have been working with face generating function, as it works the best for permutations. However, it is the best to convert back to genus generating function using the Euler formula. We thus need to compute precisely the number of vertices and edges in H-linear and H-circular families.

For a graph $H$ with $E$ edges and $V$ vertices with gluing between two sets of vertices of size $U$,

- The $H$-linear graph with $k$ copies has $kE$ edges and $k(V - U) + U$ vertices.
- The $H$-circular graph with $k$ copies has $kE$ edges and $k(V - U)$ vertices.

Therefore, we need the following transformation:

- $H$-linear case: face generating function $f(t, x)$ gives genus generating function $x^{1 + U/2} f(tx^{(E-V+U)/2}, x^{-1/2})$;
- $H$-circular case: face generating function $f(t, x)$ gives genus generating function $x f(tx^{(E-V+U)/2}, x^{-1/2})$;

In [30]:
def H_family_genus_gf(H, n, U1, U2, ftype, algorithm='pade'):
    '''
    Returns the H-linear or the H-circular genus generating function of the given graph and gluing.

    The gluing is given by identifying each pair of vertices in `U1` and `U2` with the same length.

    We offer two algorithms:

    - 'pade', which computes the first terms and then do optimized Padé's approximation for the generating function.
    - 'det', which computes the denominator explicitly, then use the first terms for the numerator.

    Both algorithms contains some heuristics, but 'det' assumes less and thus more reliable, while also taking more time.

    INPUT:

    - `H`: The graph given as a list of edges (pairs of integers from 0 to `n` - 1)
    - `n`: The number of vertices. The vertices are presented as numbers from 0 to `n` - 1.
    - `U1`: A list of vertices
    - `U2`: A list of vertices of the same length as `U1`.
    - `algorithm`: The algorithm to use, either 'pade' by default for using Padé's approximation, or 'det' with the determinant
    '''
    def half_power(poly):
        accu = 0
        tt = poly.parent().gen()
        for tpow, xpoly in poly.dict().items():
            xaccu = 0
            xx = xpoly.parent().gen()
            for xpow, xcoeff in xpoly.dict().items():
                if xpow % 2 != 0:
                    print(poly)
                    raise ValueError('Odd power, not possible to half it')
                xaccu += xx^(xpow // 2) * xcoeff
            accu += tt^tpow * xaccu
        return accu

    edgecnt = len(H)
    Ucnt = len(U1)
    numer, denom, extrapow = 0, 0, 0
    if ftype == 'linear':
        extrapow = -Ucnt
        if algorithm == 'pade':
            numer, denom = pade_heuristic_linear(H, n, U1, U2)
        elif algorithm == 'det':
            res = H_linear_gf(H, n, U1, U2)
            numer, denom = res.numerator(), res.denominator()
        else:
            raise ValueError('Invalid algorithm')
    elif ftype == 'circular':
        extrapow = 0
        if algorithm == 'pade':
            numer, denom = pade_heuristic_circular(H, n, U1, U2)
        elif algorithm == 'det':
            res = H_circular_gf(H, n, U1, U2)
            numer, denom = res.numerator(), res.denominator()
        else:
            raise ValueError('Invalid algorithm')
    else:
        raise ValueError('Invalid family type')
    # first we work with x^{1/2} to avoid fractional powers
    # Laurent polynomials are needed to express negative coefficients
    LRx = LaurentPolynomialRing(ZZ, x)
    PolyLRtx = PolynomialRing(Rx, t)
    FracLRtx = PolyLRtx.fraction_field()
    numer, denom = PolyLRtx(numer), PolyLRtx(denom)
    numer = numer.substitute(x=x^(-1), t=t * x^(edgecnt - n + Ucnt)) * LRx(x)^(2 + extrapow)
    denom = denom.substitute(x=x^(-1), t=t * x^(edgecnt - n + Ucnt))
    res = FracLRtx(numer) / FracLRtx(denom)
    numer, denom = res.numerator(), res.denominator()
    # maybe there are negative powers
    minval_numer = min(poly.valuation() for poly in numer.coefficients())
    minval_denom = min(poly.valuation() for poly in denom.coefficients())
    minval = min(minval_numer, minval_denom)
    numer *= x^(-minval)
    denom *= x^(-minval)
    numer, denom = Polytx(numer), Polytx(denom)
    # now powers in x are positive, but we cannot half the power in general
    # ugly hack
    return half_power(numer), half_power(denom)

In [31]:
def H_linear_genus_gf(H, n, U1, U2, algorithm='pade'):
    '''
    Returns the H-linear genus generating function of the given graph and gluing.

    A thin wrapper of `H_family_genus_gf`.
    '''
    return H_family_genus_gf(H, n, U1, U2, 'linear', algorithm)

def H_circular_genus_gf(H, n, U1, U2, algorithm='pade'):
    '''
    Returns the H-circular genus generating function of the given graph and gluing.

    A thin wrapper of `H_family_genus_gf`.
    '''
    return H_family_genus_gf(H, n, U1, U2, 'circular', algorithm)

In [32]:
testgfnumer, testgfdenom = H_linear_genus_gf([[0, 1]] * 3, 2, [1], [0])

Estimation phase
Series terms count: 1, time elapsed 0.0008738040924072266
Series terms count: 2, time elapsed 0.0011298656463623047
Series terms count: 3, time elapsed 0.0050127506256103516
Series terms count: 4, time elapsed 0.09510374069213867
Series terms count: 5, time elapsed 0.45055222511291504
Series terms count: 6, time elapsed 0.9119412899017334
Series terms count: 7, time elapsed 1.3886773586273193
Series terms count: 8, time elapsed 1.8738555908203125
Series terms count: 9, time elapsed 2.352273941040039
Series terms count: 10, time elapsed 2.8392066955566406
Series terms count: 11, time elapsed 3.3261144161224365
Series terms count: 12, time elapsed 3.8161001205444336
Series terms count: 13, time elapsed 4.3084821701049805
Series terms count: 14, time elapsed 4.8221540451049805
Series terms count: 15, time elapsed 5.334232807159424
Series terms count: 16, time elapsed 5.864420413970947
Series terms count: 17, time elapsed 6.430077791213989
Series terms count: 18, time elap

In [33]:
testgfnumer.factor()

(-1) * 2 * t * (12*x*t + 1) * (60*x^2*t - x - 1)

In [34]:
testgfdenom.factor()

43200*x^3*t^3 + (2880*x^2 - 1080*x)*t^2 + (-120*x - 18)*t + 1

In [35]:
testgfnumer, testgfdenom = H_circular_genus_gf([[0, 1]] * 3, 2, [1], [0])

Estimation phase
Series terms count: 1, time elapsed 0.0004010200500488281
Series terms count: 2, time elapsed 0.0022437572479248047
Series terms count: 3, time elapsed 0.0675516128540039
Series terms count: 4, time elapsed 0.4027106761932373
Series terms count: 5, time elapsed 1.094294548034668
Series terms count: 6, time elapsed 1.8996336460113525
Series terms count: 7, time elapsed 2.7033746242523193
Series terms count: 8, time elapsed 3.5246102809906006
Series terms count: 9, time elapsed 4.332018613815308
Series terms count: 10, time elapsed 5.143988370895386
Series terms count: 11, time elapsed 5.9688026905059814
Series terms count: 12, time elapsed 6.819619178771973
Series terms count: 13, time elapsed 7.67631459236145
Series terms count: 14, time elapsed 8.575678586959839
Series terms count: 15, time elapsed 9.48997950553894
Series terms count: 16, time elapsed 10.416428804397583
Series terms count: 17, time elapsed 11.394840478897095
Series terms count: 18, time elapsed 12.365

In [36]:
testgfnumer.factor()

2^3 * t * ((4353564672000000*x^13 + 6530347008000000*x^12 + 2176782336000000*x^11)*t^12 + (1741425868800000*x^12 + 4389844377600000*x^11 + 798153523200000*x^10 - 72559411200000*x^9)*t^11 + (104001822720000*x^11 + 1004947845120000*x^10 + 13907220480000*x^9 - 33105231360000*x^8 + 453496320000*x^7)*t^10 + (-45934138368000*x^10 + 77840123904000*x^9 - 26086116096000*x^8 - 4237671168000*x^7 + 292253184000*x^6)*t^9 + (-8270429184000*x^9 - 2348775014400*x^8 - 2839894732800*x^7 + 69872025600*x^6 + 48876825600*x^5 - 503884800*x^4)*t^8 + (-286738444800*x^8 - 482184161280*x^7 + 77853000960*x^6 + 37246884480*x^5 + 1266710400*x^4 - 151165440*x^3)*t^7 + (29358288000*x^7 + 3310009920*x^6 + 18235730880*x^5 - 302575824*x^4 - 186099120*x^3 - 3779136*x^2)*t^6 + (1454500800*x^6 + 1357250256*x^5 - 84107160*x^4 - 161394768*x^3 + 2420280*x^2 - 23328*x)*t^5 + (-67003200*x^5 - 25857144*x^4 - 39956652*x^3 + 2545344*x^2 + 159732*x)*t^4 + (-1441584*x^4 - 1868292*x^3 + 732510*x^2 + 81432*x + 1134)*t^3 + (102312*x^3

In [37]:
testgfdenom.factor()

(-1) * (6*x*t + 1) * (12*x*t - 1) * (43200*x^3*t^3 + (2880*x^2 - 1080*x)*t^2 + (-120*x - 18)*t + 1) * (129600*x^4*t^4 + 21600*x^3*t^3 + (180*x^2 - 144*x)*t^2 - 60*x*t + 1) * (259200*x^4*t^4 + (60480*x^3 - 2160*x^2)*t^3 + (2160*x^2 - 612*x)*t^2 + (-114*x - 6)*t + 1)